In [242]:
import xarray as xr
import stmtools

# Todo: rename to you local zarr data path.
stmat = xr.open_zarr('../data/stm.zarr')
stmat

<xarray.Dataset>
Dimensions:    (points: 318225, time: 10)
Coordinates:
    azimuth    (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
    lat        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    lon        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    range      (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points
Data variables:
    amplitude  (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    complex    (points, time) complex64 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>

In [243]:
print(stmat["lon"].shape)
print(stmat["lat"].shape)
print(stmat["amplitude"][:,0].shape)

(318225,)
(318225,)
(318225,)


In [244]:
print(stmat["lon"].values)
print(stmat["lat"].values)
print(stmat["amplitude"][:,0].values)

[4.810414 4.815807 4.818206 ... 4.962382 4.962496 4.962553]
[52.231483 52.232105 52.232384 ... 52.503086 52.5031   52.503105]
[  3300.7268  29238.707    8054.4707 ... 276654.2     64619.95
   6178.8945]


In [245]:
import laspy
import numpy as np

data_x = stmat["lon"].to_numpy()
data_y = stmat["lat"].to_numpy()
data_z = stmat["amplitude"][:,0].to_numpy()/2e+10
data_i = np.array(range(0, len(data_x))) % 65536

#data = np.hstack((data_x.reshape((-1, 1)), data_y.reshape((-1, 1)), data_z.reshape((-1, 1))))
#data = np.transpose(np.vstack((data_x.reshape((-1)), data_y.reshape((-1)), data_z.reshape((-1)))))
#data = np.vstack((data_x.reshape((-1)), data_y.reshape((-1)), data_z.reshape((-1))))
#dt = np.dtype({'names': ['X', 'Y', 'Z'], 'formats':['<f8', '<f8', '<f8']})
data = np.hstack((data_x.reshape((-1, 1)), data_y.reshape((-1, 1)), data_z.reshape((-1, 1)), data_i.reshape((-1, 1))))
dt = np.dtype({'names': ['X', 'Y', 'Z', 'Intensity'], 'formats':['<f8', '<f8', '<f8', '<u2']})

# Note that the PDAL morton order filter expects the sorting columns to be called X,Y,Z.
# We add the intensity column here to later store the indices of the points as intensity. This is strictly for visualization purposes.

data

array([[4.81041384e+00, 5.22314835e+01, 1.65036340e-07, 0.00000000e+00],
       [4.81580687e+00, 5.22321053e+01, 1.46193531e-06, 1.00000000e+00],
       [4.81820583e+00, 5.22323837e+01, 4.02723543e-07, 2.00000000e+00],
       ...,
       [4.96238184e+00, 5.25030861e+01, 1.38327096e-05, 5.60780000e+04],
       [4.96249580e+00, 5.25031013e+01, 3.23099744e-06, 5.60790000e+04],
       [4.96255302e+00, 5.25031052e+01, 3.08944720e-07, 5.60800000e+04]])

In [246]:
# Convert the array of arrays into a numpy array of tuples.

#data_l = list(zip(data[:,0], data[:,1], data[:,2])) # Alternative method.
data_l = list(map(tuple, data))
data_np = np.array(data_l,dt)
data_np

array([(4.81041384, 52.23148346, 1.65036340e-07,     0),
       (4.81580687, 52.23210526, 1.46193531e-06,     1),
       (4.81820583, 52.23238373, 4.02723543e-07,     2), ...,
       (4.96238184, 52.50308609, 1.38327096e-05, 56078),
       (4.9624958 , 52.50310135, 3.23099744e-06, 56079),
       (4.96255302, 52.50310516, 3.08944720e-07, 56080)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2')])

In [251]:
print(data.shape)
print(arr.shape)

(318225, 4)
(318225,)


In [252]:
print(np.min(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3]-np.min(data, axis=0)[0:3])

[ 4.73311424 52.23148346  0.        ]
[ 5.03681993 52.50310516  0.05570228]
[0.30370569 0.2716217  0.05570228]


In [257]:
# Write the data before sorting to las file (strictly for testing purposes).

data_write = data.copy()

# 1. Create a new header
header = laspy.LasHeader(point_format=2, version="1.2")
header.offsets = np.min(data, axis=0)[0:3]
header.scales = np.array([0.00001, 0.00001, 0.00001])

# 2. Create a Las
las = laspy.LasData(header)

las.x = data_write[:, 0]
las.y = data_write[:, 1]
las.z = data_write[:, 2]
#las.intensity = data_i
las.intensity = data_write[:, 3]

filename_raw = "/storage/MobyLe/data/test_raw.laz"
las.write(filename_raw)

In [258]:
print(np.min(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3]-np.min(data, axis=0)[0:3])

[ 4.73311424 52.23148346  0.        ]
[ 5.03681993 52.50310516  0.05570228]
[0.30370569 0.2716217  0.05570228]


In [259]:
# This is the actually important (functional) part. The rest is mostly for visualizing the results.
# You don't need to write the output to a file to work with it.


import pdal

#arr = np.array(data,dt)
arr = data_np

pipeline_morton = pdal.Filter.mortonorder().pipeline(arr)
print(pipeline_morton.execute())

arr

318225


array([(4.81041384, 52.23148346, 1.65036340e-07,     0),
       (4.81580687, 52.23210526, 1.46193531e-06,     1),
       (4.81820583, 52.23238373, 4.02723543e-07,     2), ...,
       (4.96238184, 52.50308609, 1.38327096e-05, 56078),
       (4.9624958 , 52.50310135, 3.23099744e-06, 56079),
       (4.96255302, 52.50310516, 3.08944720e-07, 56080)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2')])

In [260]:
# Write the (modulo of the) point indices to the intensity column (strictly for visulization/testing).

arr_morton = pipeline_morton.arrays[0]
arr_morton["Intensity"] = np.array(range(0, len(arr_morton))) % 65536
arr_morton["Intensity"]

arr_morton

array([(4.80885363, 52.23181152, 9.95832039e-08,     0),
       (4.80867338, 52.23242569, 5.43377610e-06,     1),
       (4.80873156, 52.23243332, 7.62824857e-06,     2), ...,
       (4.96338511, 52.50040436, 1.16414431e-05, 56078),
       (4.96342278, 52.50028229, 6.89363105e-06, 56079),
       (4.96334696, 52.50052643, 1.64161866e-05, 56080)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2')])

In [261]:
# Write the data after sorting to las file (strictly for testing purposes).


filename_morton = "/storage/MobyLe/data/test_morton.laz"
pipeline = pdal.Writer.las(
    filename=filename_morton,
    offset_x="auto",
    offset_y="auto",
    offset_z="auto",
    scale_x=0.00001,
    scale_y=0.00001,
    scale_z=0.00001,
).pipeline(arr_morton)
print(pipeline.execute())

318225


In [263]:
import laspy

las = laspy.read(filename_morton)

print(las.header)
print(las.header.offsets)
print(las.header.scales)

print(las.x)

<LasHeader(1.2, <PointFormat(3, 0 bytes of extra dims)>)>
[4.88496709e+00 5.23672943e+01 2.78511401e-02]
[1.e-05 1.e-05 1.e-05]
<ScaledArrayView([4.80885709 4.80867709 4.80872709 ... 4.96338709 4.96342709 4.96334709])>
